## Introduction
In this tutorial, we will be analyzing the performance of our FEA code. We will begin by generating a 2D mesh of a rectangle, applying initial conditions such as element type and boundaries, and finally comparing it to an analytically calculated solution.

We begin by importing all the necessary functions and their supporter functions.
We create functions which will help populate the computational imports and create a second python cell with all element initial conditions. 

To set up this example problem, lets take into consideration the construction of a stoplight, where we are focused on just the horizontal portion of the beam. Lets image this beam is fixed on the left hand side where it meets the vertical beam connected to the ground. This stoplight will then experience a heavy downward load as if a flock of very heavy birds decided to nest on it temporarily. Lets begin by defining our functions and implementing this example.

In [1]:
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis import solver_demo_helper_functions as solver_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import numpy as np

def define_sample_problem_geom(ele_type, nx, ny, L, H):
    coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)
    return coords, connect

def define_sample_problem_info(ele_type, coords, connect, L, H):
    # Identify boundaries
    boundary_nodes, boundary_edges = pre.identify_rect_boundaries(coords, connect, ele_type, 0.0, L, 0.0, H)

    # Fixed nodes on the left edge
    fixed_nodes = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)

    # Assign distributed load on the right boundary
    q = -0.20
    dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, q)

    # Assign material properties
    E = 167800.0
    nu = 0.6666666666666667
    mu = E / (2.0 * (1.0 + nu))
    kappa = E / (3.0 * (1.0 - 2.0 * nu))
    
    material_props = np.array([mu, kappa])
    # Assign artificial displacement field
    displacement = np.zeros((coords.shape))
    for kk in range(0, coords.shape[0]):
        displacement[kk, 0] = coords[kk, 0] * 0.01
    return displacement, material_props, fixed_nodes, dload_info




## Element Properties
In this next cell, we start defining our element type and our physical properties. When choosing from an element type, our options range from 1D to 2D and either tri or quad elements. You may find more information under discretization.py in the src folder. 

Once initialization is complete, we can pass this information to our created functions in the previous code cell. This will pre-generate a rectangular mesh, assign boundaries to this shape, and apply a load to a designated location. In our case as mentioned before, fixed on the left, and load applied from the top.

In [2]:
ele_type = "D2_nn4_quad"
nx = 100
ny = 8
L = 387
H = 10.0

# Generate mesh and plot it
coords, connect = define_sample_problem_geom(ele_type, nx, ny, L, H)
displacement, material_props, fixed_nodes, dload_info = define_sample_problem_info(ele_type, coords, connect, L, H)

fname = "GeneratedMeshImage.png"
pre_demo.plot_mesh_2D(fname, ele_type, coords, connect)


## Duration in Calculation
We may now test our code by create our stiffness matrix and analyzing how quickly we are able to solve it by utilizing either the dense method, or the sparse method. Since our stiffness matrix contains several hundred zeros in certain rows and columns, the sparse solver breaks down the matrix into just the usable data and essentially ignores the zeros, allowing our code to run more efficiently. Let us analyze this by running the code snippet below. We should expect the sparse matrix solver to be several magnitudes quicker than our dense solver. The more complex our elements and problem formulations become, the more efficient this sparse solver is.

In [3]:
# Testing a "dense" solver by putting together the stiffness matrix
K, R = solver_demo.prep_for_matrix_solve(ele_type, coords.T, connect.T, material_props, displacement.T, fixed_nodes, dload_info)
method = "dense"
num_runs = 5
avg_time_dense_solve = solver_demo.time_one_matrix_solve(K, R, method, num_runs)
print("average time dense matrix solve:", avg_time_dense_solve, "seconds")

# Sparse solver
method = "sparse"
num_runs = 10
avg_time_sparse_solve = solver_demo.time_one_matrix_solve(K, R, method, num_runs)
print("average time sparse matrix solve:", avg_time_sparse_solve, "seconds")

average time dense matrix solve: 0.07346773999997822 seconds
average time sparse matrix solve: 0.0026104000000486847 seconds


## Solution
Finally, let us put all of this together and pass our information to the hyperelastic solver function which will create a gif of our structure deformation. This gif may be found in the same directory as this tutorial file.

In [4]:
# run the example to look at the results

nr_num_steps = 15
nr_print = False

displacements_all, nr_info_all = hyperelastic_solver(
    material_props, 
    ele_type, 
    coords.T, 
    connect.T, 
    fixed_nodes, 
    dload_info, 
    nr_print, 
    nr_num_steps, 
    nr_tol=1e-9, 
    nr_maxit=30)

fname = "displacement.gif"
viz.make_deformation_gif(displacements_all, coords, connect, ele_type, fname)

## Answer Confirmation
It is necessary for us to compare our computed solution to an analytical one to ensure accuracy of our code. Our stoplight may be modeled as a beam with the weight of the birds applied uniformly along the entire length. We then choose a the edge of our stoplight beam which is NOT fixed, and compare this computed displacement to the mathematical one.

In [5]:
final_disp = displacements_all[-1]  # shape: (n_nodes*ndof,)

E = 167800.0
nu = 0.6666666666666667
L = 387
H = 10.0
# We need to pick a node near the end of the beam and half of the height
tip_node = 0
for i, (x, y) in enumerate(coords):
    if abs(x - L) < 1e-6 and abs(y - H/2) < H/(2*ny):
        tip_node = i
        break

tip_disp_y = final_disp[tip_node * 2 + 1]

# Compare with the analytical solution
q = -0.2
E_eff = E / (1 - nu ** 2.0)
I = H ** 3 / 12.0
w_analytical = q * L ** 4 / (8.0 * E_eff * I)

print(f"Tip node index: {tip_node}, coordinates={coords[tip_node]}")
print(f"Computed tip deflection (y): {tip_disp_y:.6f}")
print(f"Analytical Euler-Bernoulli deflection: {w_analytical:.6f}")

print(f"Error between computed vs analytical: {abs(tip_disp_y - w_analytical):.6f}")





Tip node index: 504, coordinates=[387.   5.]
Computed tip deflection (y): -22.713743
Analytical Euler-Bernoulli deflection: -22.279254
Error between computed vs analytical: 0.434489


In [ ]:
import matplotlib.pyplot as plt  
mesh_sizes = []
errors = []

# different mesh sizes to loop over
for nx, ny in [(5, 1), (10, 2), (20, 2), (30, 2), (40, 2), (40, 4), (40, 6), (50, 6), (60, 6), (60, 8), (80, 8), (100, 10)]:

    coords, connect = define_sample_problem_geom(ele_type, nx, ny, L, H)
    displacement, material_props, fixed_nodes, dload_info = define_sample_problem_info(ele_type, coords, connect, L, H)

    # solver
    displacements_all, _ = hyperelastic_solver(
        material_props, 
        ele_type, 
        coords.T, 
        connect.T, 
        fixed_nodes, 
        dload_info, 
        nr_print, 
        nr_num_steps, 
        nr_tol=1e-9, 
        nr_maxit=30)

    final_disp = displacements_all[-1]  # shape: (n_nodes*ndof,)

    # node tip and displacement
    tip_node = 0
    for i, (x, y) in enumerate(coords):
        if abs(x - L) < 1e-6 and abs(y - H/2) < H/(2*ny):
            tip_node = i
            break

    tip_disp_y = final_disp[tip_node * 2 + 1]

    # Analytical solution
    q = -0.2
    E_eff = E / (1 - nu ** 2.0)
    I = H ** 3 / 12.0
    w_analytical = q * L ** 4 / (8.0 * E_eff * I)

    # Error
    error = abs(tip_disp_y - w_analytical)

    # Storing mesh for plot
    mesh_sizes.append(nx * ny)
    errors.append(error)

# Creating a plot and saving
plt.figure()
plt.loglog(mesh_sizes, errors, marker='o', label="Error vs Mesh Size")
plt.xlabel("Mesh Size")
plt.ylabel("Error")
plt.title("Convergence to Analytical Solution")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.legend()
plt.savefig("convergence_plot.png")


c:\Users\Kote\Miniconda\envs\finite-element-analysis-env\Lib\site-packages\numpy\linalg\linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
C:\Users\Kote\AppData\Local\Temp\ipykernel_26556\2503785577.py:57: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
import matplotlib.pyplot as plt  
mesh_sizes = []
errors = []
ele_type = "D2_nn8_quad"
nr_num_steps = 15
nr_print = False


# different mesh sizes to loop over. Very small changes from previous element type
for nx, ny in [(6, 2), (10, 2), (20, 2), (30, 2), (40, 2), (40, 4), (40, 6), (50, 6), (60, 6), (60, 8), (80, 8)]:
    
    coords, connect = define_sample_problem_geom(ele_type, nx, ny, L, H)
    displacement, material_props, fixed_nodes, dload_info = define_sample_problem_info(ele_type, coords, connect, L, H)

    # solver
    displacements_all, _ = hyperelastic_solver(
        material_props, 
        ele_type, 
        coords.T, 
        connect.T, 
        fixed_nodes, 
        dload_info, 
        nr_print, 
        nr_num_steps, 
        nr_tol=1e-9, 
        nr_maxit=30)

    final_disp = displacements_all[-1]  # shape: (n_nodes*ndof,)

    # node tip and displacement
    tip_node = 0
    for i, (x, y) in enumerate(coords):
        if abs(x - L) < 1e-6 and abs(y - H/2) < H/(2*ny):
            tip_node = i
            break

    tip_disp_y = final_disp[tip_node * 2 + 1]

    # Analytical solution as before
    q = -0.2
    E_eff = E / (1 - nu ** 2.0)
    I = H ** 3 / 12.0
    w_analytical = q * L ** 4 / (8.0 * E_eff * I)

    # error computation
    error = abs(tip_disp_y - w_analytical)

    # storing mesh size and error for the plot
    mesh_sizes.append(nx * ny)
    errors.append(error)

# Plotting again
plt.figure()
plt.loglog(mesh_sizes, errors, marker='o', label="Error vs Mesh Size")
plt.xlabel("Mesh Size (Number of Elements)")
plt.ylabel("Error (|Computed - Analytical|)")
plt.title("Convergence to Analytical Solution")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.legend()
plt.savefig("convergence_plot_new_ele.png")


C:\Users\Kote\AppData\Local\Temp\ipykernel_26556\840894952.py:61: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Final Analysis
We see the calcualtions right above that show us the difference between the computed deflection of the stoplight vs the analytically calculated deflection. The "large" difference may be attributed to the "large" load that we applied. With the stoplight showing very obvious signs of bending and displacement, the materials would not be sufficient in maintaining integrity against the weight of the heavy birds as proposed in this example.